In [ ]:
#Incomplete

#remaining stuff to do:
#-implement pipelines to test using all vs. fewer features and Random Forest Regressor vs. Stacked Average model
#-add comments
#-clean up unused parts
#-maybe figure out other ways to improve score


import pandas as pd
import numpy as np
import seaborn as sns
import category_encoders as ce
from sklearn.preprocessing import OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score
from sklearn import svm
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
#from sklearn.ensemble import VotingRegressor
from sklearn import linear_model
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error

import xgboost as xgb
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import Imputer, OneHotEncoder
from sklearn.metrics import make_scorer

import timeit
import time

times = []
ttt = time.time()
def tt():
    t1 = time.time()
    td = t1 - ttt
    times.append(td)
    print('Part {} time {}'.format(len(times), td))
    ttt = t1

trainRaw = pd.read_csv('train.csv')
testRaw = pd.read_csv('test.csv')
train = trainRaw.copy()
test = testRaw.copy()

# print(train.head())
# print(test.head())

def preproc(df):
    #replace '4+' with 4 to make all Stay in City values numeric
    df['Stay_In_Current_City_Years'].replace('4+', 4, inplace=True)
    
    missing = df[['Product_Category_1', 'Product_Category_2', 'Product_Category_3']]
    missing = missing.applymap(str)
    missing = missing.replace('nan', 'missing')
    df[['Product_Category_1', 'Product_Category_2', 'Product_Category_3']] = missing
    
    #use ordinal encoder on Gender and Age columns
    ages = ['0-17', '18-25', '26-35', '36-45', '46-50', '51-55', '55+']
    mapAge = list(zip(ages, range(0,7)))
    dictGend = {'col': 'Gender', 'mapping': [('M', 0), ('F', 1)]}
    oe = ce.OrdinalEncoder(cols=['Gender', 'Age'], mapping=[dictGend, {'col': 'Age', 'mapping': mapAge}])
    df = oe.fit_transform(df)
    
    #use binary encoder on categorical features that should not be ordered
    catsNotOrd = ['User_ID', 'Product_ID', 'Occupation', 'City_Category', 'Product_Category_1', 'Product_Category_2', 'Product_Category_3']
    ceb = ce.BinaryEncoder(cols=catsNotOrd)
    df = ceb.fit_transform(df)
    
    #minmax scale Age
    scale = MinMaxScaler()
    df[['Age', 'Stay_In_Current_City_Years']] = scale.fit_transform(df[['Age', 'Stay_In_Current_City_Years']].values)
    
    return df

train = preproc(train)
test = preproc(test)

dfx = train.drop('Purchase', axis=1)
dfy = train['Purchase']

# Trying not using some features that seem to be less important
few_feat = ['Product_ID_1', 'Product_ID_2', 'Product_ID_3', 'Product_ID_4',
       'Product_ID_5', 'Product_ID_7', 'Product_ID_8', 'Product_ID_9',
       'City_Category_2', 'Product_Category_1_1', 'Product_Category_1_2',
       'Product_Category_1_3', 'Product_Category_1_4', 'Product_Category_1_5',
       'Age', 'Stay_In_Current_City_Years']
x0 = dfx.copy()
test0 = test.copy()
x1 = dfx[few_feat]
test1 = test[few_feat]

#train-test split
xtr0, xtest0, ytr0, ytest0 = train_test_split(x0, dfy, random_state=2)
xtr1, xtest1, ytr1, ytest1 = train_test_split(x1, dfy, random_state=2)

tt()

#some visualization
# f, axes = plt.subplots(3,3, figsize=(15,15))
# colCat = ['Gender', 'Age', 'Occupation', 'City_Category', 'Stay_In_Current_City_Years', 'Marital_Status',
#           'Product_Category_1', 'Product_Category_2', 'Product_Category_3']
# for a in range(0,9):
#     sns.violinplot(x=colCat[a], y='Purchase', data=trainRaw, ax=axes[int(a/3),a%3])
#
# f, ax = plt.subplots()
# sns.heatmap(trainRaw.corr())

# g = sns.PairGrid(trainRaw, y_vars="Purchase",
#                  x_vars=['Gender', 'Age', 'Occupation', 'City_Category', 'Stay_In_Current_City_Years'])
# g.map(sns.barplot)

# g = sns.PairGrid(trainRaw, y_vars="Purchase",
#                  x_vars=['Marital_Status', 'Product_Category_1', 'Product_Category_2', 'Product_Category_3'])
# g.map(sns.barplot)

rfr = RandomForestRegressor(random_state=2, n_estimators=120, n_jobs=-1)
sgdr = linear_model.SGDRegressor(random_state=2)
gbr = GradientBoostingRegressor(random_state=2)
xgbr = xgb.XGBRegressor(random_state=2)

models=[rfr, sgdr, gbr, xgbr]

# bm = models.copy()
# bstr = [0,0,0,0]
# bste = [0,0,0,0]
# for m in range(4):
#     bm[m].fit(X_train, y_train)
#     bstr[m] = bm[m].score(X_train, y_train)
#     bste[m] = bm[m].score(X_test, y_test)
#     print(bstr[m])
#     print(bste[m])

# def rmse_cv(model):
#     rmse = np.sqrt(-cross_val_score(model, dfx, dfy, scoring="neg_mean_squared_error", cv = 5))
#     return(rmse)

def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

# scores=[0,0,0,0]
# times = [0,0,0,0]
    
# def eval_score(m):
#     scores[m] = rmse_cv(models[m]).mean()
#     print(scores[m])
    
# for i in range(4):
#     times[i] = timeit.timeit('eval_score(i)', number=1, globals=globals())
#     print(times[i])
    
rfr_param_grid = {'max_features' : ['auto', 'sqrt', 'log2']}
sgdr_param_grid = {'loss' : ['squared_loss', 'huber'],
                 'penalty' : ['l2', 'l1'],
                  'alpha': [0.001, 0.0001, 0.00001]}
gbr_param_grid = {'max_depth' : [5, 7, 9]}
xgbr_param_grid = {'max_depth' : [4, 6],
                  'subsample' : [0.6, 0.8]}

#s_train = [0,0,0,0]
#s_test = [0,0,0,0]
pgrids = [rfr_param_grid, sgdr_param_grid, gbr_param_grid, xgbr_param_grid]
rs = np.zeros(4, 2)
best_estimators = np.zeros(4, 2)
b_scores = np.zeros(4, 2)
b_rmsle = np.zeros(4, 2)
pr_tr = np.zeros(4, 2)

def r_search(m):
    rs[m] = RandomizedSearchCV(models[m], param_distributions = pgrids[m], cv=5, n_jobs=-1)
    rs[m].fit(dfx, dfy)
    best_estimators[m] = rs[m].best_estimator_
    pr_tr[m] = rs[m].predict(dfx)
    
def best_score(m):
#     s_train[m] = best_estimators[m].score(X_train, y_train)
#     s_test[m] = best_estimators[m].score(X_test, y_test)
    b_scores[m] = best_estimators[m].score(dfx, dfy)
#     print(s_train[m])
#     print(s_test[m])
    print(scores[m])
    
def best_rmsle(m):
    b_rmsle[m] = rmsle(dfy, pr_tr[m])
    print(b_rmsle[m])
    
for i in range(4):
    print(i)
    r_search(i)
    best_score(m)
    best_rmsle(i)
    
feat_imp = pd.DataFrame(index=dfx.columns)
feat_imp['a'] = best_estimators[0].feature_importances_
feat_imp['b'] = best_estimators[2].feature_importances_
feat_imp['c'] = best_estimators[3].feature_importances_
feat_imp['av'] = feat_imp.mean(axis=1)
print('Feature importances')
print(feat_imp.loc[feat_imp['av']>0.005].index)
    
#copied from Kaggle kernel "Stacked Regressions to predict House Prices", by Serigne

# class AveragingModels(BaseEstimator, RegressorMixin, TransformerMixin):
#     def __init__(self, models):
#         self.models = models
        
#     # we define clones of the original models to fit the data in
#     def fit(self, X, y):
#         self.models_ = [clone(x) for x in self.models]
        
#         # Train cloned base models
#         for model in self.models_:
#             model.fit(X, y)

#         return self
    
#     #Now we do the predictions for cloned models and average them
#     def predict(self, X):
#         predictions = np.column_stack([
#             model.predict(X) for model in self.models_
#         ])
#         return np.mean(predictions, axis=1)

class StackingAveragedModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, base_models, meta_model, n_folds=5):
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_folds = n_folds
   
    # We again fit the data on clones of the original models
    def fit(self, X, y):
        self.base_models_ = [list() for x in self.base_models]
        self.meta_model_ = clone(self.meta_model)
        kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=156)
        
        # Train cloned base models then create out-of-fold predictions
        # that are needed to train the cloned meta-model
        out_of_fold_predictions = np.zeros((X.shape[0], len(self.base_models)))
        for i, model in enumerate(self.base_models):
            for train_index, holdout_index in kfold.split(X, y):
                instance = clone(model)
                self.base_models_[i].append(instance)
                instance.fit(X.iloc[train_index], y.iloc[train_index])
                y_pred = instance.predict(X.iloc[holdout_index])
                out_of_fold_predictions[holdout_index, i] = y_pred
                
        # Now train the cloned  meta-model using the out-of-fold predictions as new feature
        self.meta_model_.fit(out_of_fold_predictions, y)
        return self
   
    #Do the predictions of all base models on the test data and use the averaged predictions as 
    #meta-features for the final prediction which is done by the meta-model
    def predict(self, X):
        meta_features = np.column_stack([
            np.column_stack([model.predict(X) for model in base_models]).mean(axis=1)
            for base_models in self.base_models_ ])
        return self.meta_model_.predict(meta_features)

#end of copied code

# averaged_models = AveragingModels(models = (bests[0], bests[2], bests[3]))
# averaged_models.fit(X_train, y_train)
# pred_av = averaged_models.predict(X_train)
# score_average = rmsle(y_train, pred_av)
# print('Average:')
# print(score_average)

# test_pr_av = averaged_models.predict(X_test)
# score_test_av = rmsle(y_test, test_pr_av)
# print(score_test_av)

stacked_averaged_models = StackingAveragedModels(base_models = (best_estimators[0], best_estimators[2]),
                                                 meta_model = best_estimators[3])
# stacked_averaged_models.fit(X_train, y_train)
# pred_st_av = stacked_averaged_models.predict(X_train)
# score_st_av = rmsle(y_train, pred_st_av)
# print('Stacked average:')
# print(score_st_av)

# test_pr_st_av = stacked_averaged_models.predict(X_test)
# score_test_st_av = rmsle(y_test, test_pr_st_av)
# print(score_test_st_av)

stacked_averaged_models.fit(dfx, dfy)

pred_st_av = stacked_averaged_models.predict(dfx)

score_st_av = rmsle(dfy, pred_st_av)
print('Stacked average:')
print(score_st_av)

# predictions_f = stacked_averaged_models.predict(test)
# print(predictions_f)

# df_s = pd.DataFrame(data=testRaw[['User_ID', 'Product_ID']], index=test.index)
# df_s['Purchase'] = predictions_f
# print(df_s)
# df_s.to_csv('bfsubmission1.csv', index=False)